<a href="https://colab.research.google.com/github/MSiswanto/MultiObject_GrayImage_Detection/blob/main/multiobject_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import rospy
import cv2
import numpy as np
from sensor_msgs.msg import Image
from cv_bridge import CvBridge
from sort import Sort

# Define the maximum number of objects to track
max_objects = 10

# Create an instance of the SORT algorithm
sort = Sort(max_objects)

# Define the colors to use for each object's bounding box
colors = np.random.uniform(0, 255, size=(max_objects, 3))

# Define a deque to store the previous positions of each object
positions = {}

# Define the ROS node
rospy.init_node('object_tracker')

# Define the image transport publisher and subscriber
image_pub = rospy.Publisher('/tracked_objects', Image, queue_size=10)
image_sub = rospy.Subscriber('/image_raw', Image, image_callback)

# Define the CvBridge object
bridge = CvBridge()

def image_callback(msg):
    # Convert the ROS image message to a OpenCV image
    frame = bridge.imgmsg_to_cv2(msg, desired_encoding='bgr8')
    
    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Detect the objects in the frame
    objects = detect_objects(gray)
    
    # Sort the objects using the SORT algorithm
    objects = sort.update(objects)
    
    # Draw the bounding boxes for each object
    for i, obj in enumerate(objects):
        x, y, w, h = obj.astype(int)
        cv2.rectangle(frame, (x, y), (x + w, y + h), colors[i], 2)
        
        # Store the position of the object for tracking
        if i not in positions:
            positions[i] = deque(maxlen=30)
        positions[i].appendleft((x + w/2, y + h/2))
        
    # Draw a trail of the object's previous positions
    for i, pos in positions.items():
        for j in range(1, len(pos)):
            cv2.line(frame, pos[j], pos[j-1], colors[i], 2)
            
    # Publish the tracked image
    image_pub.publish(bridge.cv2_to_imgmsg(frame, encoding='bgr8'))

if __name__ == '__main__':
    rospy.spin()


This code assumes that you have a function called detect_objects that takes in a grayscale image and returns a list of bounding boxes for each detected object. You can use any object detection algorithm of your choice for this.

The Sort class is a Python implementation of the SORT algorithm for multi-object tracking, which takes in the maximum number of objects to track as an argument and provides an update method that takes in a list of bounding boxes and returns a new list of sorted bounding boxes.

The code also uses a dictionary called positions to store the previous positions of each object. The key of the dictionary corresponds to the index of the object in the objects list, and the value is a deque of the object's previous positions.

The image_transport package is used to publish the tracked image to a new topic called /tracked_objects. This can be viewed using a ROS image viewer or saved to a ROS bag file for later analysis.





